<a href="https://colab.research.google.com/github/ujin2021/NLP_CAMP/blob/main/day2/02_01_encoding_%EA%B0%95%EC%9D%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evn

In [2]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [3]:
# random seed initialize - random값이 균일하게 나온다
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)

# One-hot

In [ ]:
# one hot encoding text
text = """나는 책을 샀다
나는 책을 본다
나는 책을 팔았다
나는 책을 서점에서 샀다
나는 책을 도서관에서 본다"""

In [ ]:
# 띄어쓰기 단위로 split
tokens = text.split()
tokens

['나는',
 '책을',
 '샀다',
 '나는',
 '책을',
 '본다',
 '나는',
 '책을',
 '팔았다',
 '나는',
 '책을',
 '서점에서',
 '샀다',
 '나는',
 '책을',
 '도서관에서',
 '본다']

In [ ]:
# vocab 중복제거
words = list(dict.fromkeys(tokens))
words

['나는', '책을', '샀다', '본다', '팔았다', '서점에서', '도서관에서']

In [ ]:
# 각 단어에 고유한 번호 부여한 dictionary 생성
word_to_id = {'[PAD]': 0, '[UNK]': 1}  # [PAD]: 길이 맞추는 용도, [UNK]: 알 수 없는 token
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

{'[PAD]': 0,
 '[UNK]': 1,
 '나는': 2,
 '도서관에서': 8,
 '본다': 5,
 '샀다': 4,
 '서점에서': 7,
 '책을': 3,
 '팔았다': 6}

In [ ]:
# 고유한 번호로 부터 단어를 찾을 수 있는 dictionary 생성
id_to_word = {word:_id for _id, word in word_to_id.items()}
id_to_word

{0: '[PAD]',
 1: '[UNK]',
 2: '나는',
 3: '책을',
 4: '샀다',
 5: '본다',
 6: '팔았다',
 7: '서점에서',
 8: '도서관에서'}

In [ ]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

['나는 책을 샀다', '나는 책을 본다', '나는 책을 팔았다', '나는 책을 서점에서 샀다', '나는 책을 도서관에서 본다']

In [ ]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    print(sentence)
    tokens.append(sentence.split())
tokens

나는 책을 샀다
나는 책을 본다
나는 책을 팔았다
나는 책을 서점에서 샀다
나는 책을 도서관에서 본다


[['나는', '책을', '샀다'],
 ['나는', '책을', '본다'],
 ['나는', '책을', '팔았다'],
 ['나는', '책을', '서점에서', '샀다'],
 ['나는', '책을', '도서관에서', '본다']]

In [ ]:
# tokens을 vocabulary의 고유 번호로 변경
token_ids = []
for line_token in tokens:
    token_ids.append([word_to_id[token] for token in line_token])
token_ids

[[2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7, 4], [2, 3, 8, 5]]

In [ ]:
# one hot encoding
one_hot_encodings = [] # 모든 문장의 ont hot vector
for line_token in token_ids: # 문장 -> 단어 -> 단어별 id
    print(line_token)
    one_hot_line = []  # 한 줄을 표현하는 벡터
    for id in line_token: # 각 단어의 id
        one_hot = [0] * len(word_to_id) # vector길이 만큼의 onehot을 만들어 준다
        one_hot[id] = 1 # 각 단어에 해당하는 부분을 1로 변경
        print(id, one_hot)
        one_hot_line.append(one_hot)
        
    one_hot_encodings.append((one_hot_line))  # 라인을 전체 문서에 추가

[2, 3, 4]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0]
[2, 3, 5]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 1, 0, 0, 0]
[2, 3, 6]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 1, 0, 0]
[2, 3, 7, 4]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0]
7 [0, 0, 0, 0, 0, 0, 0, 1, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0]
[2, 3, 8, 5]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0]
8 [0, 0, 0, 0, 0, 0, 0, 0, 1]
5 [0, 0, 0, 0, 0, 1, 0, 0, 0]


In [ ]:
one_hot_encodings

[[[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0]]]

In [ ]:
np.argmax(np.array(one_hot_encodings[0]), axis=-1) # numpy를 array로(?) 현재 길이가 안맞아서 하나만 해본다

array([2, 3, 4])

In [ ]:
# tensorflow one hot
# token_ids가 앞 3개는 길이가 3 이지만 이후는 4로 tensorflow에서는 오류 발생 함(Can't convert non-rectangular Python sequence to Tensor.)
# depth는 vocabulary 크기
tf.one_hot(indices=token_ids, depth=len(word_to_id))

ValueError: ignored

In [ ]:
# token_ids가 앞 3개개만 one_hot으로 변경(앞의 3개는 길이가 같으므로 오류가 발생하지 않는다)
tf.one_hot(indices=token_ids[:3], depth=len(word_to_id))

<tf.Tensor: shape=(3, 3, 9), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.]]], dtype=float32)>

In [ ]:
# 모두 길이가 4가 되도록 pad(0) 추가
pad_ids = []
for line in token_ids:
    line = line[:4]
    line += [0] * (4 - len(line))
    pad_ids.append(line)
pad_ids

[[2, 3, 4, 0], [2, 3, 5, 0], [2, 3, 6, 0], [2, 3, 7, 4], [2, 3, 8, 5]]

In [ ]:
tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(word_to_id))
tf_one_hot_encodings # 3개짜리에 pad가 들어간 one hot을 넣어준다

<tf.Tensor: shape=(5, 4, 9), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]]], dtype=float32)>

# Embedding

In [ ]:
# 랜덤 매트릭스 생성 (10 ~ 99), size : 8 x 4, id별
weights = np.random.randint(10, 100, size=(len(word_to_id), 4)) / 100 
weights # float

array([[0.57, 0.93, 0.48, 0.63],
       [0.86, 0.34, 0.25, 0.59],
       [0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ],
       [0.86, 0.47, 0.44, 0.48],
       [0.77, 0.21, 0.1 , 0.85],
       [0.9 , 0.13, 0.12, 0.29],
       [0.22, 0.75, 0.85, 0.91]])

In [ ]:
# 첫번째 문장 만 numpy array로 변경
one_hot_encoding_0 = np.array(one_hot_encodings[0])
one_hot_encoding_0

array([[0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [ ]:
print(token_ids[0]) # [2, 3, 4]
print(weights[token_ids[0][0]]) # weight[2] (id 2인 단어의 random matrix : [0.33 0.36 0.4  0.53])
print(weights[token_ids[0][1]]) # weight[3]
print(weights[token_ids[0][2]]) # weight[4]

[2, 3, 4]
[0.33 0.36 0.4  0.53]
[0.4  0.36 0.68 0.79]
[0.9  0.83 0.57 0.6 ]


In [ ]:
# one hot encoding은 matrix의 특정 row를 선택하는 것과 같은 결과, matmul : 행렬곱
np.matmul(one_hot_encoding_0, weights) # one hot을 써야 한다(하지만 특별한 경우가아니면 one hot을 쓰지 않음 -> embedding, gather 을 사용)

array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]])

In [ ]:
# tensorflow 에서도 tf.keras.layers.Embedding에서도 가능 함
# weights 초기화를 위해서 [matrix] 형태로 변환함
# embdding을 사용하면 token 번호를 바로 사용가능 함 현재 표준화된 방법
embedding = tf.keras.layers.Embedding(len(word_to_id), 4, weights=[weights])

In [ ]:
embedding(np.array(token_ids[0])) # one hot vector를 만들지 않고 token id를 바로 쓸 수 있다

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]], dtype=float32)>

In [ ]:
tf.gather(weights, np.array(token_ids[0])) # embedding과 gather이 같은 역할을 한다. one  hot을 쓰지 않고 바로 구할 수 있다(보통 one hot을 만들지 않고 이렇게 사용한다)

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]])>